In [1]:
import numpy as np
import pandas as pd

In [2]:
# instructions
# rename the updated server file to 'report'
# rename Azimi's report to 'azimi'
# rename the current sheet in Azimi's reprt to 'reprot'
# run the main file.
# open the final_report excel file.
# copy and paste the content to the 'گزارش سفاله' file
# print as PDF

In [3]:
customer_codes = pd.read_excel('Excel_report/azimi.xlsx', sheet_name='کد تفصیلی', index_col=0)
Azimi_report = pd.read_excel('Excel_report/azimi.xlsx', sheet_name='report', index_col=0)
receipt = pd.read_excel('Excel_report/report.xlsx', sheet_name='Receipt', index_col=0)
delivery = pd.read_excel('Excel_report/report.xlsx', sheet_name='Delivery', index_col=0)

collective = pd.concat([receipt,delivery], axis=0)
collective = collective.sort_values('تاريخ میلادی')
collective = collective.reset_index()


In [4]:
first_day_of_the_year = ['2022-03-21','2023-03-21','2024-03-20']
not_in_azimi = []
date_filter = first_day_of_the_year + not_in_azimi
collective = collective.loc[~collective['تاريخ میلادی'].isin(date_filter),:].copy()
# collective[(collective['تاريخ میلادی'].dt.year== 2024) & (collective['تاريخ میلادی'].dt.month == 3)]

In [5]:
collective['بدهکار'] = -collective.loc[collective['مقدار (کیلوگرم)']<0,'مقدار (کیلوگرم)']
collective['بستانکار'] = collective.loc[collective['مقدار (کیلوگرم)']>0,'مقدار (کیلوگرم)']
collective = collective.fillna(0)

In [6]:
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective['بدهکار تجمیعی'] = group_by_cust['بدهکار'].cumsum()
collective['بستانکار تجمیعی'] = group_by_cust['بستانکار'].cumsum()

collective['مانده در خط'] = collective['بدهکار تجمیعی'] - collective['بستانکار تجمیعی']

In [7]:
def not_settled_inline(collective):
    collective['مانده درخط تسویه نشده'] = collective['بدهکار تجمیعی'] - collective['بستانکار'].sum()
    collective.loc[collective['مانده درخط تسویه نشده']<0,'مانده درخط تسویه نشده'] = 0
    return collective
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective = group_by_cust.apply(not_settled_inline)

In [8]:
collective['سفارش تسویه شده'] = collective['بدهکار'] - collective['مانده درخط تسویه نشده']
collective.loc[collective['سفارش تسویه شده']<0,'سفارش تسویه شده'] = 0
collective['سفارش تسویه نشده'] = collective['بدهکار'] - collective['سفارش تسویه شده']

In [9]:
report_end_date = pd.to_datetime('2024-06-28')
# report_end_date = pd.Timestamp.now()
credit_in_days = 15

In [10]:
collective['روز از ارسال سفارش تسویه نشده'] = pd.Timedelta(days=0)
collective.loc[collective['سفارش تسویه نشده']>0,'روز از ارسال سفارش تسویه نشده'] = report_end_date - collective['تاريخ میلادی']
collective['روز از ارسال سفارش تسویه نشده'] = collective['روز از ارسال سفارش تسویه نشده'].dt.days

collective['تسویه نشده × روز از ارسال'] = collective['سفارش تسویه نشده'] * collective['روز از ارسال سفارش تسویه نشده']

In [11]:
collective['تاخیر از سررسید'] = 0
collective.loc[collective['سفارش تسویه نشده']>0,'تاخیر از سررسید'] = collective['روز از ارسال سفارش تسویه نشده'] - credit_in_days
collective.loc[collective['تاخیر از سررسید']<0,'تاخیر از سررسید'] = 0

collective['سفارش سررسید شده'] = collective.loc[collective['تاخیر از سررسید']>0,'سفارش تسویه نشده']
collective['سفارش سررسید شده'] = collective['سفارش سررسید شده'].fillna(0)

collective['سررسید شده × تاخیر از سررسید'] = collective['سفارش سررسید شده'] * collective['تاخیر از سررسید']

In [12]:
collective['سررسید شده × روز از ارسال'] = collective['سفارش سررسید شده'] * collective['روز از ارسال سفارش تسویه نشده']

In [13]:
columns = pd.Series(collective.columns)
# columns

In [14]:
excel_file_path = 'Excel_report/collective.xlsx'
# collective.to_excel(excel_file_path, index=False)

# filter collective by customer
# collective[collective['کد ردیابی']==30068]

In [15]:
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
group_by_cust_sum = group_by_cust.sum(numeric_only=True)
group_by_cust_sum['متوسط زمان تسویه نشده'] = (group_by_cust_sum['تسویه نشده × روز از ارسال'] / group_by_cust_sum['سفارش تسویه نشده']).round(0).fillna(0).astype(int)
group_by_cust_sum['متوسط زمان سررسید شده'] = (group_by_cust_sum['سررسید شده × روز از ارسال'] / group_by_cust_sum['سفارش سررسید شده']).round(0).fillna(0).astype(int)
group_by_cust_sum['متوسط تاخیر از سررسید'] = (group_by_cust_sum['سررسید شده × تاخیر از سررسید'] / group_by_cust_sum['سفارش سررسید شده']).round(0).fillna(0).astype(int)

In [16]:
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)

group_by_cust_max_days = group_by_cust.agg({'روز از ارسال سفارش تسویه نشده': 'max'})
group_by_cust_max_days = group_by_cust.agg({'روز از ارسال سفارش تسویه نشده': 'max'}).rename(columns={'روز از ارسال سفارش تسویه نشده': 'بازه تسویه نشده'})
merg_sum_days = pd.merge(group_by_cust_sum,group_by_cust_max_days,on='کد ردیابی')

In [17]:
report = merg_sum_days.iloc[:,[8,18,15,12,16,17]]

In [18]:
Azimi_report = Azimi_report.iloc[:-1,:]
Azimi_report = Azimi_report.loc[Azimi_report['مانده سوفاله']<0,:]
Azimi_report.index = Azimi_report.index.astype(int)
report_filterred_by_azimi = pd.merge(Azimi_report[['عنوان','مانده سوفاله']],report,left_index=True,right_index=True,how='left')
report_filterred_by_azimi['مغایرت عظیمی'] = report_filterred_by_azimi['مانده سوفاله'] + report_filterred_by_azimi['سفارش تسویه نشده']
report_filterred_by_azimi['سفارش تسویه نشده'] = report_filterred_by_azimi['سفارش تسویه نشده'].fillna(0)
report_filterred_by_azimi['سفارش سررسید شده'] = report_filterred_by_azimi['سفارش سررسید شده'].fillna(0)
report_filterred_by_azimi = report_filterred_by_azimi.fillna('-')
report_filterred_by_azimi

,عنوان,مانده سوفاله,سفارش تسویه نشده,بازه تسویه نشده,متوسط زمان تسویه نشده,سفارش سررسید شده,متوسط زمان سررسید شده,متوسط تاخیر از سررسید,مغایرت عظیمی
كد تفصيلي,,,,,,,,,
30004,توليدي صنعتي صفاشير پارس,-11837.0,11837.0,6,3,0.0,0,0,0.0
30068,شيرگاز ايران,-121.0,0.0,0,0,0.0,0,0,-121.0
30127,شير گاز آذران,-17759.0,17759.0,18,10,2998.0,18,3,0.0
10003,اصغر نورعلي,-21540.0,21540.0,53,27,15539.0,33,18,0.0
30162,شركت الماس آريا,-132.0,132.0,65,65,132.0,65,50,0.0
30009,شركت ايمن تك پيشرو,-28957.0,28957.0,31,12,8453.0,28,13,0.0
10055,مريم فيض آبادي,-3450.0,3450.0,18,2,524.0,18,3,0.0
10027,مصطفي نورعلي,-7645.0,7645.0,13,6,0.0,0,0,0.0
10015,مصدقي,-4042.0,4042.0,9,7,0.0,0,0,0.0


In [19]:
final_report = report_filterred_by_azimi.iloc[:,[0,2,3,4,5,6,7,]]
final_report

,عنوان,سفارش تسویه نشده,بازه تسویه نشده,متوسط زمان تسویه نشده,سفارش سررسید شده,متوسط زمان سررسید شده,متوسط تاخیر از سررسید
كد تفصيلي,,,,,,,
30004,توليدي صنعتي صفاشير پارس,11837.0,6,3,0.0,0,0
30068,شيرگاز ايران,0.0,0,0,0.0,0,0
30127,شير گاز آذران,17759.0,18,10,2998.0,18,3
10003,اصغر نورعلي,21540.0,53,27,15539.0,33,18
30162,شركت الماس آريا,132.0,65,65,132.0,65,50
30009,شركت ايمن تك پيشرو,28957.0,31,12,8453.0,28,13
10055,مريم فيض آبادي,3450.0,18,2,524.0,18,3
10027,مصطفي نورعلي,7645.0,13,6,0.0,0,0
10015,مصدقي,4042.0,9,7,0.0,0,0


In [20]:
excel_file_path = 'Excel_report/final_report.xlsx'
final_report.to_excel(excel_file_path, index=False)

In [21]:
# extras to be implemented later
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective['تعداد کل روزهای بدهکار'] = group_by_cust['تاريخ میلادی'].diff()
group_by_cust = collective.groupby('کد ردیابی')
collective['تعداد کل روزهای بدهکار'] = group_by_cust['تعداد کل روزهای بدهکار'].shift(-1)

def assign_to_last(group, column):
    value = report_end_date - group.iloc[-1,group.columns.get_loc('تاريخ میلادی')]
    group.iloc[-1, group.columns.get_loc(column)] = value
    return group

group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective = group_by_cust.apply(assign_to_last,column = 'تعداد کل روزهای بدهکار')

collective['مانده در خط تسویه شده'] = collective['مانده در خط'] - collective['مانده درخط تسویه نشده']
